# Global params

In [1]:
src_directory = '/Users/rwilliams/Desktop/colbert/src'
dest_directory = '/Users/rwilliams/Desktop/colbert/dest'

# Imports and helper functions

In [96]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import skimage.exposure
from skimage.transform import resize
from skimage.util import crop
import os
%matplotlib inline

def resize_crop(img):
    crops = ((25, 439), (512, 512), (0, 0))
    cropped = crop(img, crops)
    processed = resize(cropped, (128, 128))
    processed = skimage.exposure.equalize_adapthist(processed)
    return processed

def load_img(id):
    filename = 'out%s.png' % id
    path = os.path.join(src_directory, filename)
    return sp.ndimage.imread(path)

def save_img(img, id):
    filename = '%06d.jpg' % id
    path = os.path.join(dest_directory, filename)
    sp.misc.imsave(path, img)

In [101]:
for i in range(7573):
    img = load_img(i+1)
    transf = resize_crop(img)
    save_img(transf, i+1)

/Users/rwilliams/anaconda/envs/keras/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/rwilliams/anaconda/envs/keras/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
/Users/rwilliams/anaconda/envs/keras/lib/python3.6/site-packages/skimage/exposure/exposure.py:297: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)
